In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import cv2
import os
import sys
import tensorflow as tf
import keras
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dense, Flatten, BatchNormalization, Activation, AveragePooling2D
from keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, load_img
from tensorflow.keras import regularizers
from tensorflow.image import ssim
from tensorflow.keras.applications import VGG16
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.backend import repeat_elements
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras import backend as K

In [2]:
def prepare_dataframe(input_folder):
    image_dir = input_folder
    file_path =  [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith(('JPG','PNG','JPEG','jpeg','.jpg', '.png'))]
    df = pd.DataFrame({
        'filename' : file_path,
        'class' : [0] * len(file_path)
    })
    
    return df

In [3]:
df_train=prepare_dataframe("split_10k/train") #path of train folder within split folder

In [4]:
df_test=prepare_dataframe("split_10k/test")  #path of test folder within split folder

In [5]:
df_val=prepare_dataframe("split_10k/val")   #path of val folder within split folder

In [6]:
def datagenerator(input_dataframe, img_size, batch_size):
    datagen = ImageDataGenerator(rescale=1./255)
    
    img = datagen.flow_from_dataframe(
        dataframe=input_dataframe,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='input',
        shuffle=True,
        workers=8,
        use_multiprocessing=True 
    )
    
    return img

In [7]:
img_size = 1024
batch_size = 64

In [8]:
train_img = datagenerator(df_train, img_size, batch_size)
val_img = datagenerator(df_val, img_size, batch_size)

Found 7000 validated image filenames.
Found 1500 validated image filenames.


In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_img = test_datagen.flow_from_dataframe(
        dataframe=df_test,
        target_size=(1024, 1024),
        batch_size=64,
        color_mode='grayscale',
        class_mode="input",
        shuffle=False,
        workers=8,
        use_multiprocessing=True 
    )

Found 1500 validated image filenames.


In [10]:
def custom_huber_loss(y_true, y_pred, delta=1.0, l2_lambda=0.01):
    huber_loss = tf.keras.losses.Huber(delta=delta)
    huber = huber_loss(y_true, y_pred)

    l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in autoencoder.trainable_variables if 'bias' not in v.name])
    total_loss = huber + l2_lambda*l2_loss
    return total_loss

In [11]:
def ssim_loss(y_true, y_pred, max_val=1.0):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=max_val))

In [12]:
def ms_ssim_loss(y_true, y_pred):
    ms_ssim = tf.image.ssim_multiscale(y_true, y_pred, max_val=1.0, k2=0.06, filter_size=5)
    loss = 1 - ms_ssim
    return loss

In [13]:
def ms_ssim_l1_loss(y_true, y_pred):
    ms_ssim = tf.image.ssim_multiscale(y_true, y_pred, max_val=1.0)
    ms_ssim_loss = 1 - ms_ssim
    l1_loss = tf.reduce_mean(tf.abs(y_true - y_pred))
    total_loss = 0.80*ms_ssim_loss + 0.20*l1_loss
    return total_loss

In [14]:
def grad_loss(y_true, y_pred):
    gradient_true_x, gradient_true_y = tf.image.image_gradients(y_true)
    gradient_pred_x, gradient_pred_y = tf.image.image_gradients(y_pred)
    loss_x = tf.reduce_mean(tf.abs(gradient_true_x - gradient_pred_x))
    loss_y = tf.reduce_mean(tf.abs(gradient_true_y - gradient_pred_y))
    mse=tf.reduce_mean(tf.square(y_true-y_pred))
    return loss_x + loss_y + mse

In [20]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    
    def build_autoencoder(input_img):
    
         x = Conv2D(10, (3, 3), activation='relu', padding='same')(input_img)
         x = AveragePooling2D((2, 2), padding='same')(x)

         x = Conv2D(1, (3,3), activation='relu',padding='same')(x)
         encoded = AveragePooling2D((2,2), padding='same')(x)
        
         x = Conv2D(1,(3,3), activation='relu',padding='same')(encoded) 
         x = UpSampling2D((2,2))(x)

         x = Conv2D(10, (3, 3), activation='relu', padding='same')(x)
         x = UpSampling2D((2, 2))(x)
        
         dec_output = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

         autoencoder = Model(input_img, dec_output)
         
         autoencoder.compile(optimizer='rmsprop', loss=lambda y_true, y_pred: ms_ssim_loss(y_true, y_pred))
         autoencoder.summary()
   
         return autoencoder

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [21]:
input_img = Input(shape=(1024, 1024, 1))
autoencoder = build_autoencoder(input_img)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1024, 1024, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 1024, 1024, 10) │           100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 512, 512, 10)   │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 512, 512, 1)    │            91 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_3             │ (None, 256, 256, 1)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 256, 256, 1)    │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 512, 512, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 512, 512, 10)   │           100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 1024, 1024, 10) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 1024, 1024, 1)  │            91 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 392 (1.53 KB)

 Trainable params: 392 (1.53 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
early_stopping = EarlyStopping(
             monitor='val_loss',
             patience=10,
            verbose=1,
            restore_best_weights=True       
)

In [23]:
import datetime
start_time = datetime.datetime.now()
print(start_time)

2024-09-03 12:02:20.319661


In [ ]:
history = autoencoder.fit(train_img, 
                          epochs=100, 
                          batch_size=64,
                          validation_data=val_img,
                          callbacks=[early_stopping]
                          )

Epoch 1/100


C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


110/110 ━━━━━━━━━━━━━━━━━━━━ 726s 6s/step - loss: 0.1899 - val_loss: 0.1127
Epoch 2/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 711s 6s/step - loss: 0.1016 - val_loss: 0.0753
Epoch 3/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 713s 6s/step - loss: 0.0698 - val_loss: 0.0590
Epoch 4/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 714s 6s/step - loss: 0.0567 - val_loss: 0.0529
Epoch 5/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 715s 6s/step - loss: 0.0515 - val_loss: 0.0479
Epoch 6/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 711s 6s/step - loss: 0.0476 - val_loss: 0.0466
Epoch 7/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 711s 6s/step - loss: 0.0470 - val_loss: 0.0453
Epoch 8/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 712s 6s/step - loss: 0.0451 - val_loss: 0.0440
Epoch 9/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 712s 6s/step - loss: 0.0443 - val_loss: 0.0426
Epoch 10/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 713s 6s/step - loss: 0.0427 - val_loss: 0.0409
Epoch 11/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 716s 6s/step - loss: 0.0407 - val_loss: 0.0401
Epoch 12/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
autoencoder.save("10k_2L_ms_ssim_10f(avgpool,k2=0.06,f=5).keras")

In [ ]:
end_time = datetime.datetime.now()
print(end_time)

In [ ]:
print((end_time - current_time)/3600)

In [ ]:
def plot_training(history):

    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))

    

    plt.figure()
    plt.plot(epochs, loss, 'b', label='training_loss')
    plt.plot(epochs, val_loss, 'r', label='validation_loss')
    plt.title("Training & Validation Loss")
    plt.legend()
    plt.show()

In [ ]:
plot_training(history)